In [ ]:
from __future__ import print_function

import argparse

from twitter.oauth import OAuth
from twitter import Twitter
import os
import json

In [ ]:
os.environ["https_proxy"] = os.environ["http_proxy"]

token = '2360463842-4dvRToWEoFCFkIj9Dg9fDrREZytBRhpiVan1SVl'
token_secret = 'H7xC8EBkiibjbd7yFgLlK5xRH6qq6Byxr8wp07xhwbDCk'
consumer_key = 'ogAcB1y6qPLmntrY2KCXvgggF'
consumer_secret = '9YN11mY0VWltFiOfxp6EtTu5uY8dVxHMsvMZOC3AMcb850568B'

t = Twitter(
    auth=OAuth(token, token_secret, consumer_key, consumer_secret))

import py2neo
py2neo.authenticate('localhost:7474','neo4j','neo4j')
graph = py2neo.Graph('http://localhost:7474/db/data')

In [ ]:
def create_user_node_safely(graph, screen_name, id_str=None):
    if isinstance(screen_name, str):
        screen_name = [screen_name]
        id_str = [id_str]
    
    screen_name_first = filter(lambda (x,y): y is None or y is '', zip(screen_name, id_str))
    if len(screen_name_first) > 0:
        screen_name_str = ','.join(map(lambda (x,y): '"'+x+'"', screen_name_first))
        graph.cypher.execute("""with [%s] as a1
                                foreach (i in range(0,%s) |
                                    merge (n:user {screen_name:a1[i]})
                                    )"""%(screen_name_str,len(screen_name_first)-1))
    
    screen_name_first = filter(lambda (x,y): y is not None and y is not '', zip(screen_name, id_str))
    if len(screen_name_first) > 0:
        screen_name_str = ','.join(map(lambda (x,y): '"'+x+'"', screen_name_first))
        id_str_join = ','.join(map(lambda (x,y): '"'+y+'"', screen_name_first))
        graph.cypher.execute("""with [%s] as a1, [%s] as a2
                                foreach (i in range(0,%s) |
                                    merge (n:user {screen_name:a1[i]})
                                    set n.id_str=a2[i])"""%(screen_name_str, id_str_join,len(screen_name_first)-1))


In [ ]:
def get_favorited_tweets_list(graph,screen_name,t):
    favorited = t.favorites.list(screen_name=screen_name,count = 200, skip_status=True)
    favorited_users = [(f['user']['screen_name'], f['user']['id_str']) for f in favorited]

    favorited_users_count = {fu:0 for fu in set(favorited_users)}
    for fu in favorited_users:
        favorited_users_count[fu] += 1

    unames = favorited_users_count.keys()
    sn = map(lambda x: x[0], unames)
    id_s = map(lambda x: x[1], unames)
    create_user_node_safely(graph, sn, id_s)
    for usr,cnt in favorited_users_count.iteritems():
        graph.cypher.execute("""MATCH (a {screen_name:"%s"}),(b {screen_name:"%s"})
                                MERGE (a)-[r:link]->(b)
                                SET r.favorited=%s
                                """%(screen_name, usr[0], favorited_users_count[usr]))

In [ ]:
def get_retweet_list(graph,screen_name,t):
    status_rt_list = []
    statuses = t.statuses.user_timeline(screen_name=screen_name,count = 200)
    replied_to = []
    retweeted_who = []
    for status in statuses:
        if status['retweet_count'] > 0 and 'retweeted_status' not in status:
            status_rt_list.append((screen_name, status['id_str']))
        if status['in_reply_to_screen_name']:
            replied_to.append((status['in_reply_to_screen_name'],
                               status['in_reply_to_user_id_str']))
        if 'retweeted_status' in status:
            u_id = (status['retweeted_status']['user']['screen_name'],
                    status['retweeted_status']['user']['id_str'])
            retweeted_who.append(u_id)

    s_names = map(lambda x:x[0],replied_to)
    id_s = map(lambda x:x[0],replied_to)
    create_user_node_safely(graph, s_names, id_s)

    replied_to_count = {rt:0 for rt in set(replied_to)}
    for usr in replied_to:
        replied_to_count[usr] += 1

    for usr,cnt in replied_to_count.iteritems():
        graph.cypher.execute("""MATCH (a {screen_name:"%s"}),(b {screen_name:"%s"})
                            MERGE (a)-[r:link]->(b)
                            SET r.replied=%s
                            """%(screen_name, usr[0], replied_to_count[usr]))

    s_names = map(lambda x:x[0],retweeted_who)
    id_s = map(lambda x:x[1],retweeted_who)
    create_user_node_safely(graph, s_names, id_s)

    retweeted_who_count = {rt:0 for rt in set(retweeted_who)}
    for usr in retweeted_who:
        retweeted_who_count[usr] += 1

    for usr,cnt in retweeted_who_count.iteritems():
        graph.cypher.execute("""MATCH (a {screen_name:"%s"}),(b {screen_name:"%s"})
                            MERGE (a)-[r:link]->(b)
                            SET r.retweeted=%s
                            """%(screen_name, usr[0], retweeted_who_count[usr]))
    return status_rt_list

In [ ]:
def get_profile_data_fav(screen_name):
    create_user_node_safely(graph, screen_name)
    get_favorited_tweets_list(graph,screen_name,t)
    r = graph.cypher.execute("""MATCH (a {screen_name:"%s"})-[r:link]->(p2)
                                where r.favorited > 0
                                return p2.screen_name, r.favorited"""%(screen_name,))
    user_dict = {ri[0]:ri[1] for ri in r}
    
    return user_dict

In [ ]:
def get_profile_data_conv(screen_name):
    create_user_node_safely(graph, screen_name)
    
    status_rt_list = get_retweet_list(graph,screen_name,t)

    r = graph.cypher.execute("""MATCH (a {screen_name:"%s"})-[r:link]->(p2)
                                where r.retweeted > 0
                                return p2.screen_name, r.retweeted"""%(screen_name,))
    user_dict = {ri[0]:ri[1] for ri in r}
    r = graph.cypher.execute("""MATCH (a {screen_name:"%s"})-[r:link]->(p2)
                                where r.replied > 0
                                return p2.screen_name, r.replied"""%(screen_name,))
    for ri in r:
        if ri[0] in user_dict:
            user_dict[ri[0]] += ri[1]
        else:
            user_dict.update({ri[0]:ri[1]})
    
    return status_rt_list, user_dict

In [ ]:
screen_name = 'gunjan_amit'
status_rt_list = []

user_dict = get_profile_data_fav(screen_name)

status_rt_list, user_dict2 = get_profile_data_conv(screen_name)

for k,v in user_dict2.iteritems():
    if k in user_dict:
        user_dict[k] += v
    else:
        user_dict[k] = v

user_list = [k for k,v in user_dict.iteritems() if v > 1]

In [ ]:
for i in range(len(user_list)):
    user = str(user_list[0])
    get_profile_data_fav(user)
    get_profile_data_conv(user)
    del user_list[0]